# Library Load

In [ ]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import random

# Data Load

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# X, Y split

In [ ]:
X_train = train.iloc[:, 1:-14]
Y_train = train.iloc[:, -14:]
X_test = test.iloc[:, 1:]

## modeling test for 1 column

In [ ]:
from sklearn.ensemble import RandomForestRegressor # decision tree 앙상블 모델, 배깅

 # seed 고정
user_seed = 42
random.seed(user_seed) # seed 고정

Random_Forest = RandomForestRegressor()

Random_Forest.fit(X_train, np.array(Y_train.iloc[:, 0]).reshape(-1,1))
Random_Forest.predict(X_val) # .reshape은 for 2D array

# Multi Output Regressor

## Random Forest Regressor

In [ ]:
# import numpy as np

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

# seed 고정
user_seed = 42
random.seed(user_seed) # seed 고정

regr = MultiOutputRegressor(RandomForestRegressor(random_state=user_seed)).fit(X_train, Y_train)
Y_pred = regr.predict(X_test)

Y_pred

### submission

In [ ]:
for i in range(Y_pred.shape[1]):
    submission.iloc[:,i+1] = Y_pred[:,i]

submission.shape
submission.head()

In [ ]:
submission.to_csv('RF.csv', index = False)

# 고도화

### train-val split
for hyper params tuning

In [ ]:
user_seed = 0
random.seed(user_seed) # seed 고정
from sklearn.model_selection import train_test_split

# 8:2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=user_seed)

X_train.shape
np.array(Y_train.iloc[:, 0]).reshape(-1,1).shape
np.array(Y_val.iloc[:, 0]).reshape(-1,1).shape
Y_val.iloc[:, 0].shape

## grid_search

## Multi LGBM

In [ ]:
# import numpy as np

from sklearn.multioutput import MultiOutputRegressor
# from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV

# seed 고정
user_seed = 42
random.seed(user_seed) # seed 고정

param = {
    'estimator__n_estimators': np.linspace(50, 200, 16, dtype=int),
     'estimator__max_depth': np.linspace(30, 200, 18, dtype=int),
    'estimator__learning_rate': np.linspace(0.001, 0.1, 10, dtype = float),
    'estimator__min_child_weight': np.linspace(0.001, 0.1, 10, dtype=float)
    
}

gs = GridSearchCV(estimator=MultiOutputRegressor(lgbm.LGBMRegressor()), 
                  param_grid=param, 
                  cv=2,
                  scoring = 'neg_mean_squared_error',
                  n_jobs = 16,
                 verbose = 1)

regr = gs.fit(X_train, Y_train)

print(regr.best_score_, regr.best_params_)
regr_model = regr.best_estimator_


regr_model # 출력시 best_param_가 끼인 상태의 모델이 출력됨.

# 제목으로 쓸만한 부분 (best_params_가 모델안에 들어간 형태로 출력됨.)
regr_model.get_params()['estimator']

In [ ]:
from sklearn.metrics import mean_squared_error as mse # 모델 평가 지표 scoring (mse)
from sklearn.metrics import r2_score as r2
Y_val_pred = regr_model.predict(X_val)
mse(Y_val, Y_val_pred)
# valid 스코어가 잘 나왔다면 동일 모델로 X_test 예측 돌리기

In [ ]:
Y_pred = regr_model.predict(X_test)

### 파일 제출 형태

In [ ]:
for i in range(Y_pred.shape[1]):
    submission.iloc[:,i+1] = Y_pred[:,i]

submission.shape
submission.head()

In [ ]:
submission.to_csv(f'grid {regr_model.get_params()['estimator']}.csv', index = False)